## Q2

Setup

In [39]:
# import required packages
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
# make it easier to understand by importing the required libraries within keras
from tensorflow.keras.layers import Dense, Flatten
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## Q2.a

Data file IO

In [40]:
df = pd.read_csv("randomized_data.csv")
print(df.shape)

(177, 14)


Load and normalize traing and test data

In [41]:
# column 1 = label, column 2 -> 14 = input features
(x, y) = df.iloc[:, 1:14], df.iloc[:, 0]

# normalization
n_x = MinMaxScaler().fit_transform(x)

# 75% for training, 25% for testing 
x_train, x_test, y_train, y_test = train_test_split(n_x, y, test_size=0.25)

print('Training set shape: {}'.format(x_train.shape))
print('Training labels shape: {}'.format(y_train.shape))
print('Test set shape: {}'.format(x_test.shape))
print('Test label shape: {}'.format(y_test.shape))


Training set shape: (132, 13)
Training labels shape: (132,)
Test set shape: (45, 13)
Test label shape: (45,)


Create and convert labels to categorical

In [42]:
labels = set(y_train)

if len(y_train.shape) == 1:
    y_train = keras.utils.to_categorical(y_train-1, num_classes=3)
    y_test = keras.utils.to_categorical(y_test-1, num_classes=3)
    
print('The classes are: {}'.format(labels))
print('The values in the input data range from {} to {}'.format(x_train.min(),x_train.max()))
    

The classes are: {1, 2, 3}
The values in the input data range from 0.0 to 1.0


Function for training and testing model, here I used categorical crossentropy as loss function since there are more than 2 label classes. Epoch is set to 100 to prevent underfitting. Finally, I used the test accuracy as metric for the model. Since the classification is simple enough, cross validation is not implemented here

In [43]:
def fit(model, x_train, y_train, x_test, y_test):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    h = model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=0)

    # return test accurancy 
    return model.evaluate(x_test,y_test, verbose=0)[1]

Function for generating a 4x4 grid of accuracy values comparing different combination of neurons and hidden layers

In [44]:
def generateGrid(data):
    # create the dataframe
    df = pd.DataFrame(data, columns=["4", "16", "32", "64"],
                    index=["1", "2", "3", "4"])

    # print the dataframe
    print(df)

Classification Model. Here I tested 16 different combinations of number of nodes and number of layers to find the best possible accuracy performance and used sigmoid as  activation function. 

In [45]:
n_neurons = [4, 16, 32, 64]
n_layers = [1, 2, 3, 4]

M = np.empty((len(n_neurons), len(n_layers)), dtype=object)
accuracy = np.empty((len(n_neurons), len(n_layers)), dtype=float)

for i in range(len(n_neurons)):
    for j in range(len(n_layers)):
        # model initialization
        M[i][j] = keras.models.Sequential()
        # input layer + 1st hidden layer 
        M[i][j].add(Dense(n_neurons[i], activation='sigmoid', input_shape=(13,)))
        # hidden layers
        for layer in range(n_layers[j]-1):
            M[i][j].add(Dense(n_neurons[i], activation='sigmoid'))
        # output layer 
        M[i][j].add(Dense(3, activation='softmax'))
        # train
        accuracy[i,j] = fit(M[i, j], x_train, y_train, x_test, y_test)

generateGrid(accuracy)          

          4        16        32        64
1  0.977778  0.955556  0.933333  0.600000
2  0.977778  0.955556  0.977778  0.977778
3  0.977778  0.955556  0.977778  0.977778
4  0.977778  0.977778  1.000000  1.000000


According to the table, the best possible classification performance for accuracy is 100% for both 32 neurons X 4 layers and 64 neurons X 4 layers, since both models are complex enough to ensure that the model is not underfitted. Interestingly the model is not overfitted as well, this could be because the model is still sufficiently simple due to the small layer count. 

## Q2.b

Here I used the model with 64 neurons X 4 layers to predict the classification of the 3 products

In [47]:
test = np.array([[13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285],
                   [12.04, 4.3, 2.38, 22, 80, 2.1, 1.75, 0.42, 1.35, 2.6, 0.79, 2.57, 580],
                   [14.13, 4.1, 2.74, 24.5, 96, 2.05, 0.76, 0.56, 1.35, 9.2, 0.61, 1.6, 560]])

n_test = MinMaxScaler().fit_transform(test)

prediction = M[3,3].predict(n_test)

print(list(map(lambda x: list(map(int, map(round, x))), prediction)))

1/1 [==============================] - 0s 59ms/step
[[1, 0, 0], [0, 1, 0], [0, 0, 1]]


The classification result of the 3 products are: [0,1,2] 